In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import pickle
with open("itos.pkl",'rb') as f:
    itos=pickle.load(f)
with open("stoi.pkl",'rb') as f:
    stoi=pickle.load(f)
    
vocab_size = len(itos)
embedding_size = 400

In [3]:
rnn0 = nn.LSTM(400, 1150, 1)

rnn1 = nn.LSTM(1150, 1150, 1)

rnn2 = nn.LSTM(1150, 400, 1)

rnns = nn.ModuleList([rnn0,rnn1,rnn2])

embedder= nn.Embedding(vocab_size,embedding_size)
decoder = nn.Linear(embedding_size,vocab_size,bias=False)

In [4]:
class LangModel(nn.Module):
    
    def __init__(self, embedder, rnns, decoder):
        super(LangModel, self).__init__()
        self.embedder = embedder
        self.rnns = rnns
        self.decoder = decoder
        
    def forward(self,input):
        out=embedder(input)
        for rnn in rnns:
            out,hid = rnn(out)
        out = decoder(out[:,-1])
        return out

In [5]:
model = LangModel(embedder,rnns,decoder)

In [6]:
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))
def generate(length,creativity,dist=False):
    next_word = "."
    text = ""
    for i in range (length):
        tensor_output = model(torch.tensor([[stoi[next_word]]],dtype=torch.long,device="cuda"))[0]
        output = (tensor_output).detach().cpu().numpy()
        
#hard cutoff
#         subdist = softmax(np.sort(output)[-creativity:])
#         print (subdist)
#         next_word = itos2[np.random.choice(np.argsort(output)[-creativity:],p=subdist)]

#soft cutoff
        distribution = softmax(output/creativity)
        ranks = np.argsort(distribution)
        if dist:
            print([itos[rank] for rank in ranks[-10:]])
            distribution.sort()
            print (distribution[-10:])
            break
        next_word = itos[np.random.choice(range(distribution.shape[0]),p=distribution)]
        text+=(next_word+" ")
    return text
        

In [8]:
model.load_state_dict(torch.load("./creepy_pasta.weights"))

In [9]:
model.cuda()

LangModel(
  (embedder): Embedding(257874, 400)
  (rnns): ModuleList(
    (0): LSTM(400, 1150)
    (1): LSTM(1150, 1150)
    (2): LSTM(1150, 400)
  )
  (decoder): Linear(in_features=400, out_features=257874, bias=False)
)

In [10]:
generate(100,0.5)

'" " " i have any other than a long , i was n’t have been a bit , and he had been on the house to be able to make it was n’t want to the edge of the last way . the best to me , and the small in the room , and the rest of the bottom of the room . i was a bit of the thing i was a few moments was n’t do n’t have to be in the door . the door , so much . the counter , but the love '

In [11]:
generate(100,0.25)

'i was n’t know that he had been a few days . " i was a little of the wall . the same . i had been the room . i was the door , and i was a few minutes , and i was a few seconds , i was a few minutes , i was n’t know what i was n’t have been a few minutes , but i was the first , and i was n’t want to the door . the same . i was n’t think it was n’t know that i could n’t know '

In [11]:
generate(100,0.1)

i was a few minutes . i was a few minutes . i was n’t know that i was a few minutes . i was a few minutes , and i was a few minutes , and i was a few minutes . i was a few minutes . i was a few minutes . " i was n’t know what i was a few minutes . i could n’t know what i was a few minutes . i was a few minutes , i was a few seconds . " i was a few minutes , and i was 

In [12]:
generate(100,0.75)

i am in my face . i ’d come through a movie to talk . i could never seen , i was all . i go in the job and then , " i knew that ’s just always move . i ’m not a woman from the other bed . part of the n’t have been the new or there were doing you in some days , i ’ve been in the damn was handed another and a explosion , that i ’m going to your spine . the far living room and then , but to his ears 

In [13]:
generate(100,1)

another room - were those who lordhowea mall’ , sithole , i told us towards the hell well heracleides . i would have heard out with his skin . their feletti and i got a large , and in the usual lawn words that the nearest , then you know no one that the air ’s that that right back to do n’t do . even had gone did . " she is at his flesh i only tired and more . they are each monster pillow to crush a little brière on the front of a grasp , it 

In [14]:
generate(100,0.8)

so did n’t explain it was an fire . i was being pointed . the events in my first , we had to rain , not was always wanted to be true , okay . his face in this was . all that close coming from my chest ; something stopped , the man shut a loud . the first time i had found his hotel different . that ’s a knife , and look at least as i must have to almost . still as possible , but he ’s head and a bit thing , who come back 

In [19]:
generate(100,0.85)

'that concentrated and worry of the heart of hope i realized you ca n’t looked back , and stared in from the darkness . " there " the same unseen my eyes to went more sleep . eventually , the room , " he was n’t know . " what the ground in strange and i had left in . as though he yet even been dirty was n’t the way he was … " no . i ’ve been finally as a couple of death . i picked up , a few seconds and natural . another in the '

In [29]:
expected_length = 50000
doc_length = 0
start_crazy = 0.5
end_crazy = 1
with open("novel.txt","w") as f:
    crazification_rate = (end_crazy-start_crazy)/expected_length
    while doc_length < expected_length:
        par_length = randint(50,250)
        crazy = start_crazy+doc_length*crazification_rate
        f.write(generate(par_length,crazy)+"\n\n")
        doc_length += par_length
        print (doc_length,crazy)
        

95 0.5
299 0.50095
461 0.50299
541 0.50461
677 0.50541
851 0.50677
911 0.50851
965 0.50911
1114 0.50965
1279 0.51114
1432 0.51279
1559 0.51432
1764 0.51559
1931 0.51764
2161 0.51931
2272 0.52161
2452 0.52272
2640 0.52452
2797 0.5264
3012 0.52797
3149 0.53012
3275 0.53149
3326 0.5327500000000001
3459 0.53326
3709 0.53459
3845 0.53709
3946 0.53845
4168 0.53946
4254 0.54168
4472 0.54254
4662 0.54472
4837 0.54662
4901 0.54837
5145 0.54901
5368 0.55145
5497 0.55368
5684 0.55497
5904 0.55684
6045 0.55904
6151 0.56045
6304 0.56151
6500 0.56304
6580 0.565
6744 0.5658
6960 0.5674399999999999
7113 0.5696
7215 0.57113
7437 0.57215
7619 0.57437
7722 0.57619
7913 0.5772200000000001
8039 0.57913
8108 0.58039
8201 0.58108
8371 0.58201
8558 0.58371
8715 0.58558
8792 0.5871500000000001
8857 0.58792
8943 0.58857
9005 0.58943
9159 0.59005
9379 0.5915900000000001
9441 0.59379
9530 0.59441
9624 0.5953
9853 0.59624
10030 0.59853
10228 0.6003000000000001
10388 0.60228
10635 0.60388
10708 0.6063500000000001
1

/home/sam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in less


13431 0.63181
13594 0.63431
13745 0.63594
13866 0.6374500000000001
13940 0.63866
14081 0.6394
14331 0.64081
14383 0.64331
14506 0.64383
14719 0.64506
14862 0.64719
14931 0.64862
15154 0.64931
15336 0.65154
15565 0.65336
15643 0.6556500000000001
15882 0.6564300000000001
16127 0.65882
16324 0.66127
16555 0.66324
16652 0.66555
16823 0.66652
17010 0.66823
17170 0.6701
17297 0.6717
17504 0.6729700000000001
17752 0.67504
17811 0.67752
17955 0.67811
18200 0.67955
18364 0.682
18480 0.68364
18694 0.6848000000000001
18807 0.68694
18901 0.68807
19134 0.68901
19349 0.6913400000000001
19473 0.69349
19681 0.6947300000000001
19814 0.69681
19901 0.69814
19979 0.69901
20125 0.69979
20256 0.70125
20502 0.7025600000000001
20713 0.70502
20916 0.70713
21140 0.70916
21298 0.7114
21348 0.7129800000000001
21442 0.71348
21573 0.71442
21783 0.71573
21952 0.71783
22052 0.71952
22204 0.72052
22270 0.72204
22436 0.7227
22504 0.72436
22739 0.72504
22956 0.72739
23015 0.72956
23245 0.7301500000000001
23362 0.73245
2

/home/sam/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


30687 0.8047500000000001
30909 0.80687
30988 0.8090900000000001
31162 0.80988
31407 0.81162
31621 0.8140700000000001
31719 0.8162100000000001
31926 0.8171900000000001
32075 0.8192600000000001
32189 0.8207500000000001
32370 0.82189
32436 0.8237000000000001
32638 0.82436
32797 0.82638
32973 0.8279700000000001
33054 0.8297300000000001
33153 0.8305400000000001
33375 0.8315300000000001
33438 0.83375
33617 0.83438
33755 0.8361700000000001
33876 0.83755
33964 0.83876
34201 0.83964
34378 0.84201
34435 0.84378
34610 0.84435
34685 0.8461000000000001
34741 0.8468500000000001
34888 0.84741
34956 0.8488800000000001
35084 0.8495600000000001
35269 0.85084
35408 0.85269
35528 0.85408
35616 0.85528
35816 0.85616
35909 0.85816
35987 0.85909
36114 0.85987
36197 0.86114
36447 0.86197
36665 0.8644700000000001
36821 0.86665
36888 0.86821
37105 0.8688800000000001
37266 0.8710500000000001
37483 0.87266
37653 0.87483
37844 0.87653
37900 0.8784400000000001
37958 0.879
38145 0.87958
38271 0.8814500000000001
3837

In [22]:
from random import randint

In [23]:
randint(50,250)

213